In [8]:
# !gdown 1scuFwqh8s7KIYAfZW1Eu6088ZAK2SI-v

In [9]:
# !ls ./esd_dataset/Emotion\ Speech\ Dataset

In [10]:
DS_DIR = "./esd_dataset/Emotion Speech Dataset"

import os

# os.listdir(DS_DIR + "/0003" )

In [11]:
def build_dict(ds_dir):
    d = []
    for i, dir in enumerate(os.listdir(ds_dir)):
        base_dir = os.path.join(ds_dir, dir)
        dic_file = os.path.join(ds_dir, dir, f"{dir}.txt")
        if os.path.exists(dic_file):
            d.append({
                "base_dir": base_dir,
                "dict_file": dic_file,
            })
    return d

build_dict(DS_DIR)[0]['dict_file']

'./esd_dataset/Emotion Speech Dataset/0003/0003.txt'

In [12]:
import pandas as pd

# info = build_dict(DS_DIR)[0]

def get_list_files(info):
    emotions = ['Happy', 'Sad', 'Surprise', 'Neutral', 'Angry']
    list_files = []
    for emotion in emotions:
        list_files += [os.path.join(info['base_dir'], emotion, f) for f in os.listdir(os.path.join(info['base_dir'], emotion)) if f.endswith(".wav")]
    return list_files

# get_list_files(info)[:5]

In [13]:
list_files = []

for info in build_dict(DS_DIR):
    _list_files = get_list_files(info)
    list_files += _list_files

In [14]:
import audio as Audio


In [15]:
from torch.utils.data import Dataset
import torch
import librosa

class ESDataset(Dataset):
    def __init__(self, list_files):
        self.list_files = list_files
        self.STFT = Audio.stft.TacotronSTFT(
            1024, # config["preprocessing"]["stft"]["filter_length"],
            256, # config["preprocessing"]["stft"]["hop_length"],
            1024, # config["preprocessing"]["stft"]["win_length"],
            80, # config["preprocessing"]["mel"]["n_mel_channels"],
            22050, # config["preprocessing"]["audio"]["sampling_rate"],
            0, # config["preprocessing"]["mel"]["mel_fmin"],
            8000 # config["preprocessing"]["mel"]["mel_fmax"],
        )
        self.emo2idx = {
            'Neutral': 0,
            'Happy': 1,
            'Sad': 2,
            'Angry': 3,
            'Surprise': 4
        }

    def __len__(self):
        return len(self.list_files)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        file = self.list_files[idx]
        emotion = file.split("/")[-2]
        
        wav, _ = librosa.load(file, sr=16000)
        # convert 16k to 22.05k
        wav = librosa.resample(wav, 16000, 22050)        
        mel_spectrogram, energy = Audio.tools.get_mel_from_wav(wav, self.STFT)
        
        return mel_spectrogram.T, emotion, file

In [23]:
dataset = ESDataset(list_files)
# dataset.__getitem__(0)[0].shape
# dataset.__getitem__(0)[1]
# for i in range(12600, 12700):
    # dataset.__getitem__(i)[2]
# len(dataset)

/tmp/ipykernel_33484/544894725.py:37: FutureWarning: Pass orig_sr=16000, target_sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  wav = librosa.resample(wav, 16000, 22050)


In [17]:
# !rm -rf ./esd_dataset_processed
# !mkdir -p ./esd_dataset_processed

In [32]:
import pickle as pk

# for sample in dataset:
#     mel, emo, file = sample
#     file_name = file.split("/")[-1].split(".")[0]
#     pk.dump(mel, open(f"./esd_dataset_processed/mel_{file_name}_{emo.lower()}.pkl", "wb"))
    
import multiprocessing as mp
import tqdm
import pickle as pk

def process_sample(sample):
    mel, emo, file = sample
    if not file.endswith(".wav"):
        return
    file_name = file.split("/")[-1].split(".")[0]
    new_file_name = f"./esd_dataset_processed/mel_{file_name}_{emo.lower()}.pkl"
    if os.path.exists(new_file_name):
        return
    pk.dump(mel, open(new_file_name, "wb"))

for idx in tqdm.tqdm(range(12000, len(dataset))):
    sample = dataset.__getitem__(idx)
    process_sample(sample)

# with mp.Pool(8) as pool:
#     list(tqdm.tqdm(pool.imap(process_sample, dataset), total=len(dataset)))


  0%|          | 0/23000 [00:00<?, ?it/s]/tmp/ipykernel_33484/544894725.py:37: FutureWarning: Pass orig_sr=16000, target_sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  wav = librosa.resample(wav, 16000, 22050)
100%|██████████| 23000/23000 [27:50<00:00, 13.77it/s]
